### Operational Space Control (Khatib)

1. The controller is defined as $\Gamma = \sum_{k=1}^N J_{k|prec(k)}^T F_{k|prec(k)}$
2. The prioritized jacobian for task $k$ is given by $J_{k|prec(k)} := J_k N_{prec(k)}$
3. $N_{prec(k)} = I - \sum_{i=1}^{k-1} \bar J_{i|prec(i)} J_{i|prec(i)}$
4. $N_{prec(k)} = N_{k-1|prec(k-1)} N_{k-2|prec(k-2)} ... I$. See *sentis A.24*. This is equivalent to $N_{prec(k)} = N_{k-1|prec(k-1)} N_{prec(k-1)}$

Algorithm
1. Initialize $N_{prec(1)} = I$, $priority = 1$
2. $N_{1|prec(1)} \leftarrow (I-\bar J_1 J_1)$
3. For $t_i \in T$
<br> a) If $priority(t_i) > priority$,
<br> $$N_{prec(i)} = N_{i-1|prec(i-1)} N_{prec(i-1)}$$
<br> $$priority = priority(t_i)$$ 
<br> b) Set $N_{prec(i)}$ in current task
<br> c) Calculate $J_{i|prec(i)} := J_i N_{prec(i)}$
<br> d) Calculate $N_{i|prec(i)} := (I-\bar J_{i|prec(i)} J_{i|prec(i)})$

### Hierarchical Quadratic Programming (Mansard)

1. The heart of operational space control is the implicit constrained minimization of the kinetic energy that is embedded within the dynamically consistent jacobian inverse $\bar J$. We can appreciate this more clearly with the use of lagrange multipliers applied to the following problem: $$\min_{\dot q} \frac{1}{2} \dot q^T M \dot q$$ $$st. \dot x = J \dot q$$
2. The lagrangian is given by $L = \dot q^T M \dot q - \lambda^T (\dot x - J \dot q)$ giving the partial derivatives $\frac{\partial L}{\partial \dot q} = M \dot q - J^T \lambda = 0$ and the trivial constraint equation $\frac{\partial L}{\partial \dot \lambda} = \dot x - J \dot q = 0$. Premultiplying the first equation by $J$ and substituting the second, we have $J \dot q = (J M^-1 J^T) \lambda = \Lambda^{-1} \lambda \rightarrow \lambda = \Lambda \dot x$. Putting this result back to the first equation, we have $\dot q = M^{-1} J^T \Lambda \dot x = \bar J \dot x$.